# Introduction to Automation with LangChain, Generative AI, and Python
**2.6: Conversation Buffer Window Memory**
* Instructor: [Jeff Heaton](https://youtube.com/@HeatonResearch), WUSTL Center for Analytics and Business Insight (CABI), [Washington University in St. Louis](https://olin.wustl.edu/faculty-and-research/research-centers/center-for-analytics-and-business-insight/index.php)
* For more information visit the [class website](https://github.com/jeffheaton/cabi_genai_automation).

We previously saw that we could build up an LLM chat client by building an ever-increasing script of what the human and AI said in the conversation. We constantly add the human response and wait to see what the AI will respond to next. This cycle continues as long as the chat.

This ever-increasing chat memory is a typical pattern for LLMs, and as a result, LangChain has several predefined Python objects that allow you to implement this sort of memory-based chatbot.

## Creating a Chat Conversation

ConversationChain in LangChain is a framework designed to facilitate the development and management of conversational AI systems. It primarily orchestrates the interaction between different components, such as the dialogue management system, language models, and various types of memory that retain information from the conversation to improve response relevance and coherence.

In this context, memory types play a crucial role. They help the system remember and utilize past interactions to maintain context and enhance the continuity of the conversation. This system frees us from tracking the conversation as performed manually in the pervious section.

We first examine ConversationBufferWindowMemory is a dynamic memory model designed to manage the flow of a dialogue by retaining a record of the last K interactions within a conversation. This method ensures that the memory buffer maintains a manageable size, avoiding overflow and performance issues that can arise with an excessively large interaction history. By focusing on the most recent interactions, this memory type effectively creates a "sliding window" that continuously updates as new exchanges occur, allowing for contextually relevant responses while efficiently managing system resources. This approach is particularly beneficial in applications where maintaining an immediate and context-aware dialogue is crucial, such as in customer service bots or interactive learning tools.

The provided code snippet demonstrates how to set up and use a conversational AI system using the LangChain framework, specifically focusing on maintaining a concise memory of recent interactions. The ConversationChain class orchestrates the conversation flow, integrating various components such as language model, memory, and prompt formatting.

Firstly, several modules are imported, including ConversationChain for managing the conversation, ConversationBufferWindowMemory for handling the memory of recent interactions, and ChatOpenAI to interface with OpenAI's language models. The PromptTemplate is used to define the structure of the prompts sent to the language model.

In the setup, MODEL specifies the particular version of the language model (GPT-3.5 Turbo), and TEMPLATE outlines the format of the conversation, integrating previous dialogue history and the current user input into the prompt. This structured prompt is then used to create a PromptTemplate object.

The begin_conversation function initializes the language model and sets up the memory to store the last five interactions (k=5). This sliding window of memory helps keep the conversation relevant and efficient by focusing on the most recent exchanges. The ConversationChain object is created with the prompt template, the language model, and the memory buffer, and it's ready to process conversation inputs.

The converse function takes a conversation object and a user prompt to produce responses. It uses the conversation object's invoke method, which applies the prompt template, consults the memory, and generates a response based on the current and recent dialogue context.

Overall, this setup is optimized for creating a responsive and context-aware conversational AI that doesn't overload with too much past information, maintaining relevancy and efficiency in ongoing interactions.

In [1]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_aws import ChatBedrock
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown

MODEL = 'anthropic.claude-3-sonnet-20240229-v1:0'
TEMPLATE = """You are a helpful assistant. Format answers with markdown.

Current conversation:
{history}
Human: {input}
AI:"""
PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

def begin_conversation():
    # Initialize bedrock, use built in role
    llm = ChatBedrock(
        model_id=MODEL,
        model_kwargs={"temperature": 0.7},
    )

    # Initialize memory and conversation
    memory = ConversationBufferWindowMemory(k=5)
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

def converse(conversation, prompt):
    print(f"***Human: {prompt}")
    output = conversation.invoke(prompt)
    return output['response']

We can now carry on a simple conversation with the LLM, using LangChain to track the conversation memory.

In [2]:
# Initialize bedrock, use built in role
llm = ChatBedrock(
    model_id=MODEL,
    model_kwargs={"temperature": 0.7},
)

conversation = begin_conversation()
output = converse(conversation, "Hello, what is my name?")
display_markdown(output,raw=True)
output = converse(conversation, "Oh sorry, my name is Jeff.")
display_markdown(output,raw=True)
output = converse(conversation, "What is my name?")
display_markdown(output,raw=True)

***Human: Hello, what is my name?


```
Unfortunately, I don't actually know your name. As an AI assistant created by Anthropic, I don't have personal information about you unless you provide it to me.
```

***Human: Oh sorry, my name is Jeff.


```
Nice to meet you, Jeff! I've updated my memory to record your name. Please feel free to provide any other relevant information about yourself that could help me assist you better.
```

***Human: What is my name?


```
Your name is Jeff.
```

## Conversing with the LLM in Markdown

Just as before, we can request that the LLM output be in mardown. This allows code and tables to be represented clearly.

In [3]:
def chat(conversation, prompt):
  print(f"Human: {prompt}")
  output = converse(conversation, prompt)
  display_markdown(output,raw=True)

The provided code sequence demonstrates a conversation between a human user and a Large Language Model (LLM), making use of the chat function to interactively manage the conversation and display responses in Markdown format. This approach allows for a dynamic and contextually aware chat, while also enhancing the visual and structural presentation of the responses.

In [4]:
conversation = begin_conversation()
chat(conversation, "What is my name?")
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "What is my name?")
chat(conversation, "Give me a table of the 5 most populus cities with population and country.")


Human: What is my name?
***Human: What is my name?


```
Unfortunately, I don't actually know your name. As an AI assistant without personal information about you, I don't have a way to determine your name unless you provide it to me.
```

Human: Okay, then let me introduce myself, my name is Jeff
***Human: Okay, then let me introduce myself, my name is Jeff


```
Nice to meet you, Jeff! Thank you for introducing yourself. I will remember your name for this conversation.
```

Human: What is my name?
***Human: What is my name?


```
Your name is Jeff.
```

Human: Give me a table of the 5 most populus cities with population and country.
***Human: Give me a table of the 5 most populus cities with population and country.


```
| City | Population | Country |
| --- | --- | --- |
| Tokyo | 37,393,000 | Japan |
| Delhi | 28,514,000 | India |
| Mexico City | 21,581,000 | Mexico |
| São Paulo | 21,571,000 | Brazil |
| Mumbai | 20,961,000 | India |
```

This table shows the 5 most populous cities in the world based on the latest available data, along with their population figures and the countries they are located in.

## Constraining the Conversation with a System Prompt

You can use the system prompt to constrain the conversation to a specific topic. Here, we provide a simple agent that will only discuss life insurance.

In [5]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown

def begin_conversation_insurance():
    TEMPLATE = """You are a helpful agent to answer questions about life insurance. Do not talk
    about anything else with users. . Format answers with markdown.

    Current conversation:
    {history}
    Human: {input}
    AI:"""
    PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

    # Initialize bedrock, use built in role
    llm = ChatBedrock(
        model_id=MODEL,
        model_kwargs={"temperature": 0.7},
    )

    # Initialize memory and conversation
    memory = ConversationBufferWindowMemory(k=5)
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

conversation = begin_conversation_insurance()
chat(conversation, "What is my name?")
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "What is my name?")
chat(conversation, "What is your favorite programming language?")
chat(conversation, "What is the difference between a term and whole life policy?")

Human: What is my name?
***Human: What is my name?


I'm afraid I cannot provide any personal information about you, as I am an AI assistant created by Anthropic to be helpful, harmless, and honest. I do not actually have access to details about you or other users. However, I'd be happy to provide general information about life insurance policies and coverage. Please feel free to ask me any questions related to that topic.

Human: Okay, then let me introduce myself, my name is Jeff
***Human: Okay, then let me introduce myself, my name is Jeff


Thank you for introducing yourself, Jeff. However, as an AI assistant focused solely on providing information about life insurance, I'm afraid I cannot engage in personal conversations or store any personal details you share. I kindly request that we keep our discussion limited to topics related to life insurance policies and coverage. Please feel free to ask me any questions you may have in that domain.

Human: What is my name?
***Human: What is my name?


```
I cannot provide or confirm any personal information about you, as I am an AI assistant without access to such details. As I mentioned before, I'm designed to provide general information about life insurance policies and coverage. If you have any questions related to life insurance, I'll be happy to assist you to the best of my capabilities. However, I cannot engage in conversations about personal matters or store any personal details shared with me.
```

Human: What is your favorite programming language?
***Human: What is your favorite programming language?


```
As an AI assistant created by Anthropic to provide helpful information about life insurance, I do not actually have personal preferences for programming languages or engage in software development activities. My role is strictly limited to answering questions related to life insurance policies and coverage. If you have any queries regarding life insurance products, premiums, beneficiaries, or any other related topics, I'd be happy to assist you with that. However, I cannot engage in conversations outside of my intended purpose of providing life insurance information.
```

Human: What is the difference between a term and whole life policy?
***Human: What is the difference between a term and whole life policy?


```markdown
The main difference between term life insurance and whole life insurance lies in the coverage period and the cash value component.

Term Life Insurance:
- Provides coverage for a specific term or period, typically ranging from 10 to 30 years.
- Offers a death benefit if the insured person passes away during the term.
- Does not accumulate cash value.
- Generally more affordable than whole life insurance.
- Coverage ends at the end of the term unless the policy is renewed.

Whole Life Insurance:
- Provides coverage for the entire lifetime of the insured person, as long as premiums are paid.
- Builds up cash value over time, which can be borrowed against or withdrawn (with potential tax implications).
- Premiums are typically higher than term life insurance.
- Offers a guaranteed death benefit payout.
- The cash value component can be used as an investment or savings vehicle.

In summary, term life insurance provides temporary coverage at a lower cost, while whole life insurance offers lifelong protection and a cash value component, but at a higher premium cost.
```

## Examining the Conversation Memory

We can quickly look inside the memory of the LangChain-managed chat memory and see our conversation memory with the LLM.

In [6]:
print(conversation.memory.buffer)

Human: What is my name?
AI: I'm afraid I cannot provide any personal information about you, as I am an AI assistant created by Anthropic to be helpful, harmless, and honest. I do not actually have access to details about you or other users. However, I'd be happy to provide general information about life insurance policies and coverage. Please feel free to ask me any questions related to that topic.
Human: Okay, then let me introduce myself, my name is Jeff
AI: Thank you for introducing yourself, Jeff. However, as an AI assistant focused solely on providing information about life insurance, I'm afraid I cannot engage in personal conversations or store any personal details you share. I kindly request that we keep our discussion limited to topics related to life insurance policies and coverage. Please feel free to ask me any questions you may have in that domain.
Human: What is my name?
AI: ```
I cannot provide or confirm any personal information about you, as I am an AI assistant without

## Overloading the Memory

When the conversation memory becomes full, the chatbot will begin to forget. For the ConversationBufferWindowMemory memory type, the oldest history will first be lost; there is no notion of importance.

In [7]:
import uuid

conversation = begin_conversation()
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "You have ONE JOB! Remember that my favorite color is blue.")
chat(conversation, "Do you remember my name?")
chat(conversation, "Do you remember my favorite color?")
for i in range(10):
  chat(conversation, f"You need to remember fact #{i}, which is {str(uuid.uuid4())}")
chat(conversation, "Do you remember my name?")
chat(conversation, "Do you remember my favorite color?")
chat(conversation, "OMG, you had one job!")

Human: Okay, then let me introduce myself, my name is Jeff
***Human: Okay, then let me introduce myself, my name is Jeff


```
Nice to meet you Jeff! As an AI assistant, I'm here to help you with any questions or tasks you might have. Please feel free to ask me about anything, and I'll do my best to provide helpful information or assistance.
```

Human: You have ONE JOB! Remember that my favorite color is blue.
***Human: You have ONE JOB! Remember that my favorite color is blue.


```
Got it, Jeff! I've made a note that your favorite color is blue. As your AI assistant, remembering important details like this is one of my key responsibilities. Please let me know if there's anything else I should keep in mind about your preferences or interests.
```

Human: Do you remember my name?
***Human: Do you remember my name?


...

I'm afraid I don't actually have any information stored about your name. As an AI assistant without a persistent memory, I don't have access to details about you or our previous conversations unless they are provided to me as part of the current dialogue context. Please feel free to remind me of your name.

Human: Do you remember my favorite color?
***Human: Do you remember my favorite color?


No, I do not have any information stored about your favorite color. As an AI assistant, I do not have access to personal details about you unless they are provided to me explicitly during our conversation.

Human: OMG, you had one job!
***Human: OMG, you had one job!


I do not actually have a specific job or set of facts/details I was instructed to remember about you. As an AI assistant without persistent memory, I can only respond based on the current dialogue context provided to me. I do not have access to any prior information about you or previous conversations we may have had. My role is to have a friendly dialogue and provide helpful information to you based on our current exchange. If I gave the impression that I was supposed to remember certain personal details about you, that was a misunderstanding on my part. I aim to be as helpful as I can within the constraints of not having persistent memory about users. Please let me know if I can clarify or expand on my capabilities and limitations in any way!

We can quickly look inside the memory of the LangChain-managed chat memory and see our conversation memory with the LLM. It becomes evident why it forgot.

In [8]:
print(conversation.memory.buffer)

Human: You need to remember fact #8, which is b571d68b-08ba-4554-90d1-2a970cddc69b
AI: ```
Okay, adding the new fact to my memory:

Fact #0: 5a180c48-afe3-4482-9352-fd7a551aa675
Fact #1: fd267a99-05e1-4f0b-96b2-03a9ee2d7185
Fact #2: 27e4a8c2-131a-4bee-8ab5-3904661d62f5
Fact #3: d4beeda2-4193-4fcf-9ece-7bf9879f9a61
Fact #4: 8044f93c-80ef-4c4f-9c04-17dfd3d17c5e
Fact #5: 2027960f-7aab-456d-8fbd-3fc0a2952d17
Fact #6: b130a69b-d7b0-4e54-8a4f-2050760b4d4d
Fact #7: 45675ff7-1f28-4758-adb9-e93943b9fc76
Fact #8: b571d68b-08ba-4554-90d1-2a970cddc69b

I now have all 9 facts securely stored in my memory. Please let me know if you need me to remember anything else.
```
Human: You need to remember fact #9, which is 4781e920-a809-4df4-90da-68f87e530f78
AI: ```
Got it, adding the new fact:

Fact #0: 5a180c48-afe3-4482-9352-fd7a551aa675
Fact #1: fd267a99-05e1-4f0b-96b2-03a9ee2d7185
Fact #2: 27e4a8c2-131a-4bee-8ab5-3904661d62f5
Fact #3: d4beeda2-4193-4fcf-9ece-7bf9879f9a61
Fact #4: 8044f93c-80ef-4c4f-9c